In [3]:
from  google.colab import drive, files

In [4]:
drive.mount('/content/MyDrive/')

Drive already mounted at /content/MyDrive/; to attempt to forcibly remount, call drive.mount("/content/MyDrive/", force_remount=True).


In [5]:
%cd '/content/MyDrive/MyDrive/Hand_gesture/'
print("Driver loaded Successfull")

/content/MyDrive/MyDrive/Hand_gesture
Driver loaded Successfull


In [15]:
from keras.models import Sequential
from keras.layers import Conv2D , MaxPool2D , Flatten ,Dense , MaxPooling2D
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping , ModelCheckpoint
from keras.optimizers import Adam

In [7]:
train_data_gen = ImageDataGenerator(
   rescale = 1./255,
   shear_range = 0.2,
   zoom_range = 0.2,
   rotation_range=30,
   width_shift_range=0.1,
   height_shift_range=0.1,
   horizontal_flip=True,
   fill_mode="nearest"
)


val_data_gen = ImageDataGenerator(
    rescale = 1./255
)
print("[INFO] datagenearator loaded successfully")

[INFO] datagenearator loaded successfully


In [8]:
EPOCHS = 50
INIT_LR = 1e-3

In [9]:
training_set = train_data_gen.flow_from_directory('Hand_dataset_short/train' ,
                                                  target_size = (256 , 256),
                                                  batch_size = 20,
                                                  color_mode  = 'grayscale',
                                                  classes = ['NONE','ONE','TWO','THREE','FOUR','FIVE'],
                                                  class_mode = 'categorical')

Found 600 images belonging to 6 classes.


In [10]:
val_set = val_data_gen.flow_from_directory('Hand_dataset_short/valid' ,
                                            target_size = (256 , 256),
                                            batch_size = 20,
                                            color_mode  = 'grayscale' ,
                                            classes = ['NONE','ONE','TWO','THREE','FOUR','FIVE'],
                                            class_mode = 'categorical')

Found 150 images belonging to 6 classes.


**#model loading...**

In [13]:
model = Sequential()
#1 CONV >> RELU >> POOL LAYER
model.add(Conv2D(32,(3, 3), input_shape = (256, 256, 1), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
#2 CONV >> RELU >> POOL LAYER
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
#3 CONV >> RELU >> POOL LAYER
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
#4 CONV >> RELU >> POOL LAYER
model.add(Conv2D(256, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
#ANN
model.add(Flatten())
model.add(Dense(units = 150, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 6, activation = 'softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 123, 123, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 61, 61, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 59, 59, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 29, 29, 128)      

In [16]:
opt = Adam(learning_rate= INIT_LR , decay=INIT_LR / EPOCHS)
print("[INFO]  loading optimizer")

[INFO]  loading optimizer


In [17]:
model.compile(
    optimizer = opt , 
    loss = 'categorical_crossentropy' , 
    metrics = ['accuracy']
)

In [18]:
callback_list = [
    EarlyStopping(monitor='val_loss',patience=10),
    ModelCheckpoint(filepath="model.h5",monitor='val_loss',save_best_only=True,verbose=1)]

In [ ]:
model.fit_generator(
    training_set, 
    steps_per_epoch = 10,
    epochs = EPOCHS , 
    validation_data = val_set , 
    validation_steps = 2 ,
    callbacks = callback_list
)
print('[INFO]  succesful')

In [21]:
ret , accuracy = model.evaluate(val_set  , verbose = 1)
print(accuracy*100)

8/8 [==============================] - 7s 907ms/step - loss: 1.7118 - accuracy: 0.6600
66.00000262260437


**#Testing**

In [27]:
from keras.preprocessing import image
import os
from keras.models import load_model
import numpy as np

In [28]:
model = load_model('model.h5')
print("Model Loaded Successfully")

Model Loaded Successfully


In [30]:
def testing(img_file):
    img_name = img_file
    test_image = image.load_img(img_name, target_size = (256, 256),grayscale=True)
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = model.predict(test_image)
    arr = np.array(result[0])
    print(arr)

    maxx = np.amax(arr)   #maximum of flattend array
    max_prob = arr.argmax(axis=0)  #Only the first occurrence is returned if changes happend, 0 mean 1st row
    #max_prob = max_prob + 1
    classes=["NONE", "ONE", "TWO", "THREE", "FOUR", "FIVE"]
    result = classes[max_prob]  # or result = classes[max_prob - 1]
    print(img_name,result)

In [31]:
path = 'Hand_dataset_short/test'
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
   for file in f:
     if '.png' in file:
       files.append(os.path.join(r, file))

In [32]:
for f in files:
   testing(f)
   print('\n')

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


[1. 0. 0. 0. 0. 0.]
Hand_dataset_short/test/NONE_0.png NONE


[0. 0. 1. 0. 0. 0.]
Hand_dataset_short/test/ONE_0.png TWO


[0. 0. 1. 0. 0. 0.]
Hand_dataset_short/test/TWO_0.png TWO


[0. 0. 0. 0. 1. 0.]
Hand_dataset_short/test/THREE_0.png FOUR


[0. 0. 0. 0. 1. 0.]
Hand_dataset_short/test/FOUR_0.png FOUR


[0. 0. 0. 0. 0. 1.]
Hand_dataset_short/test/FIVE_0.png FIVE


